In [112]:
import requests
import json
import pandas as pd
import numpy as np
import datetime

In [113]:
def prettify(obj):
    return json.dumps(obj, sort_keys=True, indent=2)

In [121]:
username = input('Enter username: ')
password = input('Enter password: ')
headers = {'Content-Type': 'application/json'}                  ````````````

In [122]:
with open ("api_key.json") as f:
    api_key = json.load(f)["mitchell"]

In [123]:
def get_data(name, start_time, end_time, interval):
    base_url = 'https://itsnt2259.iowa.uiowa.edu/piwebapi/search/query?q=name:'
    url = base_url + name
    query = requests.get(url, auth=(username, password), headers=headers).json()
    self_url = query['Items'][0]['Links']['Self']
    point = requests.get(self_url, auth=(username, password), headers=headers).json()
    data_url = point['Links']['InterpolatedData']
    data_url = data_url + '/?startTime=-' + start_time + '&endTime=-' + end_time + '&interval=' + interval
    data = requests.get(data_url, auth=(username, password), headers=headers).json()
    return data

In [124]:
# EXAMPLE USAGE OF get_data()
data = get_data('PP_Electric_Gen', '2w', '1w', '1d')

In [125]:
# Main Natural Gas
# Names
main_ng_names = ['PP_TB1_2_TB1_GAS_FLOW', 'HBLR_GAS_FLOW', 'PP_B7_Gas_Flow_Adj', 'PP_B8_Gas_Flow_Adj', 'PP_B10_FLT_235_FT', 'PP_BLR12_FT_006_KSCFH',
                 'PP_GG1_FUEL_FLOW', 'PP_GG2_FUEL_FLOW', 'PP_GG3_FUEL_FLOW', 'PP_GG4_FUEL_FLOW', 'PP_AF-XI-8220A']

# Main Purch El
main_purch_el_names = ['PP_Electric_Purch']

# Main Gen El
main_gen_el_names = ['PP_Electric_Gen']

# Blr 10 Pellets
blr_10_pellets_names = ['PP_CHS_B10WeighBelt_MvgAvg']

# Blr 11 Coal + Pellets
blr_11_coal_pellets_names = ['PP_SF-WIT-6044A']

# Blr 11 Oat Hulls
blr_11_oat_hulls_names = ['PP_BIO_Weight']

ALL_DATA = {'Main Natural Gas': main_ng_names, 'Main Purch El': main_purch_el_names, 'Main Gen El': main_gen_el_names,
            'Blr 10 Pellets': blr_10_pellets_names, 'Blr 11 Coal + Pellets': blr_11_coal_pellets_names, 'Blr 11 Oat Hulls': blr_11_oat_hulls_names}

# TODO Oakdale Nat Gas Est. using steam and NG Monthly BTU Factors
# TODO Oakdale Purch El
# TODO Oakdale Gen El

In [126]:
# Get the data
interval = '1h'
startTime = '1d'
endTime = '1h'

def get_df(names, startTime, endTime, interval):
    df = pd.DataFrame()
    for i, name in enumerate(names):
        data = get_data(name, startTime, endTime, interval)
        items = data['Items']
        if i == 0:
            times = [datetime.datetime.strptime(item['Timestamp'][:-2], '%Y-%m-%dT%H:%M:%S.%f') for item in items]
            df['Time'] = times
        vals = [max(item['Value'], 0) for item in items]
        df[name] = pd.DataFrame(vals)
    return df

In [127]:
main_ng_df = get_df(main_ng_names, startTime, endTime, interval)
main_purch_el_df = get_df(main_purch_el_names, startTime, endTime, interval)
main_gen_el_df = get_df(main_gen_el_names, startTime, endTime, interval)
blr_10_pellets_df = get_df(blr_10_pellets_names, startTime, endTime, interval)
blr_11_coal_pellets_df = get_df(blr_11_coal_pellets_names, startTime, endTime, interval)
blr_11_oat_hulls_df = get_df(blr_11_oat_hulls_names, startTime, endTime, interval)

In [128]:
# CO2 Emission Calculations
HEAT_CONVERSION_COEFFS = { 'natural_gas': 1.026, 'pellets': 20.89375, 'oat_hulls': 8.25, 'coal': 24.93 }
CO2_CONVERSION_COEFFS = {  'natural_gas': 53.06, 'pellets': 31.875, 'oat_hulls': 118.17, 'coal': 93.28}
PURCH_CO2_COEFF = 611.169

In [133]:
# add columns 1,2,3,4,6,11 of main_ng_df to main_ng_df['Sum']
main_ng_df['Sum'] = main_ng_df.iloc[:, 1:5].sum(axis=1) + main_ng_df.iloc[:, 6:12].sum(axis=1)
# add columns 5,7,8,9,10 of main_ng_df to main_ng_df['CO2']
main_ng_df['Sum'] += (main_ng_df.iloc[:, 5:6].sum(axis=1) + main_ng_df.iloc[:, 7:11].sum(axis=1))/1000

main_ng_df['CO2'] = main_ng_df['Sum'] * HEAT_CONVERSION_COEFFS['natural_gas'] * CO2_CONVERSION_COEFFS['natural_gas']
print("Natural Gas", main_ng_df['CO2'].head())

Natural Gas


0     172.153141
1     174.017863
2     171.161150
3    1430.175403
4    1964.453204
Name: CO2, dtype: float64

In [132]:
# calculate CO2 emissions from blr_11_oat_hulls_df
blr_11_oat_hulls_df['CO2'] = blr_11_oat_hulls_df['PP_BIO_Weight'] * HEAT_CONVERSION_COEFFS['oat_hulls'] * CO2_CONVERSION_COEFFS['oat_hulls']
print("Oat Hulls", blr_11_oat_hulls_df['CO2'].head())

0    18348.566640
1    19114.864572
2    19071.400007
3    18210.942871
4    17444.406087
Name: CO2, dtype: float64

In [130]:
# calculate CO2 emissions from blr_10_pellets_df
blr_10_pellets_df['CO2'] = blr_10_pellets_df['PP_CHS_B10WeighBelt_MvgAvg'] * HEAT_CONVERSION_COEFFS['pellets'] * CO2_CONVERSION_COEFFS['pellets']
blr_10_pellets_df['CO2'].head()

0    4996.925392
1    5464.867406
2    5034.511566
3    5628.059842
4    5625.951990
Name: CO2, dtype: float64

In [131]:
# calculate CO2 emissions from main_purch_el_df
main_purch_el_df['CO2'] = main_purch_el_df['PP_Electric_Purch'] * PURCH_CO2_COEFF
main_purch_el_df['CO2'].head()

0    23192.768274
1    22916.571346
2    22380.052239
3    20510.860426
4    19262.193693
Name: CO2, dtype: float64